## Analysis of FF Cerebellum wiht SpaGCN

In [19]:
import os,csv,re
import pandas as pd
import numpy as np
import numba
import scanpy as sc
import math
import SpaGCN as spg
from scipy.sparse import issparse
import random, torch
import warnings
warnings.filterwarnings("ignore")
import matplotlib.colors as clr
import matplotlib.pyplot as plt
import SpaGCN as spg
#In order to read in image data, we need to install some package. Here we recommend package "opencv"
#inatll opencv in python
#!pip3 install opencv-python
import cv2

In [20]:
##Read in preprocessed adata
#Read in gene expression and spatial location
adata=sc.read("ff_human_cerebellum.h5ad")

#read in spatial positions
spatial=pd.read_csv("Parent_Visium_Human_Cerebellum/spatial/tissue_positions_list.csv",sep=",",header=None,na_filter=False,index_col=0,names=["barcode", "in_tissue", "array_row", "array_col","pxl_row_in_fullres","pxl_col_in_fullres"]) 

#Read in hitology image
img=cv2.imread("Parent_Visium_Human_Cerebellum/spatial/tissue_hires_image.tiff")


In [21]:
##re index spatial table according to barcodes in adata
sorted_barcodes = list(adata.obs_names)

spatial = spatial.reindex(sorted_barcodes)

In [23]:
#Set coordinates
x_array=adata.obs["array_row"].tolist()
y_array=adata.obs["array_col"].tolist()
x_pixel=spatial["pxl_row_in_fullres"].tolist()
y_pixel=spatial["pxl_col_in_fullres"].tolist()

print(img)
#Test coordinates on the image
img_new=img.copy()
for i in range(len(x_pixel)):
    x=x_pixel[i]
    y=y_pixel[i]
    img[int(x-20):int(x+20), int(y-20):int(y+20),:]=0

cv2.imwrite('ff_cerebellum_scanpy_processed_image_map.jpg', img_new)

[[[184 182 179]
  [184 182 179]
  [183 183 179]
  ...
  [130 130 130]
  [  5   5   5]
  [  0   0   0]]

 [[183 182 179]
  [184 182 179]
  [184 183 179]
  ...
  [130 131 130]
  [  5   5   5]
  [  0   0   0]]

 [[183 183 178]
  [184 182 179]
  [184 183 180]
  ...
  [130 131 129]
  [  5   5   5]
  [  0   0   0]]

 ...

 [[  0   0   0]
  [  0   0   0]
  [  0   0   0]
  ...
  [  0   0   0]
  [  0   0   0]
  [  0   0   0]]

 [[  0   0   0]
  [  0   0   0]
  [  0   0   0]
  ...
  [  0   0   0]
  [  0   0   0]
  [  0   0   0]]

 [[  0   0   0]
  [  0   0   0]
  [  0   0   0]
  ...
  [  0   0   0]
  [  0   0   0]
  [  0   0   0]]]


True

In [24]:
#Calculate adjacent matrix
s=1
b=49
#print(img)
#adj=spg.calculate_adj_matrix(x=x_pixel,y=y_pixel, x_pixel=x_pixel, y_pixel=y_pixel, image=img, beta=b, alpha=s, histology=True)
#If histlogy image is not available, SpaGCN can calculate the adjacent matrix using the fnction below
adj=spg.calculate_adj_matrix(x=x_pixel,y=y_pixel, histology=False)
print(adj)
np.savetxt('adj.csv', adj, delimiter=',')

Calculateing adj matrix using xy only...
[[   0.     6461.73   6142.252  ... 4243.298  5103.237  6237.3794]
 [6461.73      0.      553.3263 ... 2898.7065 2499.441  5195.0464]
 [6142.252   553.3263    0.     ... 2393.8943 1949.4218 5462.931 ]
 ...
 [4243.298  2898.7065 2393.8943 ...    0.      861.0023 6144.1704]
 [5103.237  2499.441  1949.4218 ...  861.0023    0.     6515.515 ]
 [6237.3794 5195.0464 5462.931  ... 6144.1704 6515.515     0.    ]]


In [25]:
#set hyper-parameters
p=0.5 
#Find the l value given p
l=spg.search_l(p, adj, start=0.01, end=1000, tol=0.01, max_run=100)
print(l)

Run 1: l [0.01, 1000], p [0.0, 121.99695776175334]
Run 2: l [0.01, 500.005], p [0.0, 37.32079315185547]
Run 3: l [0.01, 250.0075], p [0.0, 11.132019996643066]
Run 4: l [0.01, 125.00874999999999], p [0.0, 2.9255335330963135]
Run 5: l [62.509375, 125.00874999999999], p [0.336236834526062, 2.9255335330963135]
Run 6: l [62.509375, 93.7590625], p [0.336236834526062, 1.4576241970062256]
Run 7: l [62.509375, 78.13421875], p [0.336236834526062, 0.838670015335083]
Run 8: l [62.509375, 70.321796875], p [0.336236834526062, 0.568342924118042]
Run 9: l [66.4155859375, 70.321796875], p [0.44661617279052734, 0.568342924118042]
recommended l =  68.36869140625001
68.36869140625001


In [26]:
#Going with 14 clusters as this is what I got from previous data - this has 0 index
n_clusters=10
#Set seed
r_seed=t_seed=n_seed=100
#Search for suitable resolution
res=spg.search_res(adata, adj, l, n_clusters, start=0.7, step=0.1, tol=5e-3, lr=0.05, max_epochs=20, r_seed=r_seed, t_seed=t_seed, n_seed=n_seed)

Start at res =  0.7 step =  0.1
Initializing cluster centers with louvain, resolution =  0.7
Epoch  0
Epoch  10
Res =  0.7 Num of clusters =  6
Initializing cluster centers with louvain, resolution =  0.7999999999999999
Epoch  0
Epoch  10
Res =  0.7999999999999999 Num of clusters =  7
Res changed to 0.7999999999999999
Initializing cluster centers with louvain, resolution =  0.8999999999999999
Epoch  0
Epoch  10
Res =  0.8999999999999999 Num of clusters =  8
Res changed to 0.8999999999999999
Initializing cluster centers with louvain, resolution =  0.9999999999999999
Epoch  0
Epoch  10
Res =  0.9999999999999999 Num of clusters =  8
Res changed to 0.9999999999999999
Initializing cluster centers with louvain, resolution =  1.0999999999999999
Epoch  0
Epoch  10
Res =  1.0999999999999999 Num of clusters =  10
recommended res =  1.0999999999999999


In [27]:
clf=spg.SpaGCN()
clf.set_l(l)
#Set seed
random.seed(r_seed)
torch.manual_seed(t_seed)
np.random.seed(n_seed)
#Run
clf.train(adata,adj,init_spa=True,init="louvain",res=res, tol=5e-3, lr=0.05, max_epochs=200)
y_pred, prob=clf.predict()
adata.obs["pred"]= y_pred
adata.obs["pred"]=adata.obs["pred"].astype('category')
#Do cluster refinement(optional)
#shape="hexagon" for Visium data, "square" for ST data.
adj_2d=spg.calculate_adj_matrix(x=x_array,y=y_array, histology=False)
refined_pred=spg.refine(sample_id=adata.obs.index.tolist(), pred=adata.obs["pred"].tolist(), dis=adj_2d, shape="hexagon")
adata.obs["refined_pred"]=refined_pred
adata.obs["refined_pred"]=adata.obs["refined_pred"].astype('category')

Initializing cluster centers with louvain, resolution =  1.0999999999999999
Epoch  0
Epoch  10
Epoch  20
Epoch  30
Epoch  40
Epoch  50
Epoch  60
Epoch  70
Epoch  80
Epoch  90
Epoch  100
Epoch  110
Epoch  120
Epoch  130
Epoch  140
Epoch  150
Epoch  160
Epoch  170
Epoch  180
Epoch  190
Calculateing adj matrix using xy only...


In [28]:
#Plot spatial domains
#Set colors used
plot_color=["#F56867","#FEB915","#C798EE","#59BE86","#7495D3","#D1D1D1","#6D1A9C","#15821E","#3A84E6","#997273","#787878","#DB4C6C","#9E7A7A","#554236","#AF5F3C","#93796C","#F9BD3F","#DAB370","#877F6C","#268785"]
#Plot spatial domains
domains="pred"
num_celltype=len(adata.obs[domains].unique())
adata.uns[domains+"_colors"]=list(plot_color[:num_celltype])
#change above line so it comes from adata input
ax=sc.pl.scatter(adata,alpha=1,x="array_row",y="array_col",color=domains,title=domains,color_map=plot_color,show=False,size=100000/adata.shape[0])
ax.set_aspect('equal', 'box')
ax.axes.invert_yaxis()
plt.savefig("ff_cerebellum_spagcn_pred.png", dpi=600)
plt.close()

#Plot refined spatial domains
domains="refined_pred"
num_celltype=len(adata.obs[domains].unique())
adata.uns[domains+"_colors"]=list(plot_color[:num_celltype])
ax=sc.pl.scatter(adata,alpha=1,x="array_row",y="array_col",color=domains,title=domains,color_map=plot_color,show=False,size=100000/adata.shape[0])
ax.set_aspect('equal', 'box')
ax.axes.invert_yaxis()
plt.savefig("ff_cerebellum_spagcn_refined_pred.png", dpi=600)
plt.close()

In [29]:
#Use domain 0 as an example
target=0
#Set filtering criterials
min_in_group_fraction=0.8
min_in_out_group_ratio=1
min_fold_change=1.5
#Search radius such that each spot in the target domain has approximately 10 neighbors on average
adj_2d=spg.calculate_adj_matrix(x=x_array, y=y_array, histology=False)
start, end= np.quantile(adj_2d[adj_2d!=0],q=0.001), np.quantile(adj_2d[adj_2d!=0],q=0.1)
r=spg.search_radius(target_cluster=target, cell_id=adata.obs.index.tolist(), x=x_array, y=y_array, pred=adata.obs["pred"].tolist(), start=start, end=end, num_min=10, num_max=14,  max_run=100)
r = 3.1622776601683795
#Detect neighboring domains
print(r)
nbr_domians=spg.find_neighbor_clusters(target_cluster=target,
                                   cell_id=adata.obs.index.tolist(), 
                                   x=adata.obs["array_row"].tolist(), 
                                   y=adata.obs["array_col"].tolist(), 
                                   pred=adata.obs["pred"].tolist(),
                                   radius=r,
                                   ratio=1/2)


print(nbr_domians)
nbr_domians=nbr_domians[0:3]

de_genes_info=spg.rank_genes_groups(input_adata=adata,
                                target_cluster=target,
                                nbr_list=nbr_domians, 
                                label_col="pred", 
                                adj_nbr=True, 
                                log=True)
#Filter genes
de_genes_info=de_genes_info[(de_genes_info["pvals_adj"]<0.05)]
filtered_info_domain_0=de_genes_info
filtered_info_domain_0=filtered_info_domain_0[(filtered_info_domain_0["pvals_adj"]<0.05) &
                            (filtered_info_domain_0["in_out_group_ratio"]>min_in_out_group_ratio) &
                            (filtered_info_domain_0["in_group_fraction"]>min_in_group_fraction) &
                            (filtered_info_domain_0["fold_change"]>min_fold_change)]
filtered_info_domain_0=filtered_info_domain_0.sort_values(by="in_group_fraction", ascending=False)
filtered_info_domain_0["target_dmain"]=target
filtered_info_domain_0["neighbors"]=str(nbr_domians)
print("SVGs for domain ", str(target),":", filtered_info_domain_0["genes"].tolist())

Calculateing adj matrix using xy only...
Calculateing adj matrix using xy only...
Calculateing adj matrix using xy only...
Run 1: radius [1.4142135381698608, 16.552946090698242], num_nbr [1.0, 178.57777777777778]
Calculateing adj matrix using xy only...
Run 2: radius [1.4142135381698608, 8.983579814434052], num_nbr [1.0, 73.15555555555555]
Calculateing adj matrix using xy only...
Run 3: radius [1.4142135381698608, 5.198896676301956], num_nbr [1.0, 30.466666666666665]
Calculateing adj matrix using xy only...
Run 4: radius [1.4142135381698608, 3.3065551072359085], num_nbr [1.0, 15.1]
Calculateing adj matrix using xy only...
Run 5: radius [2.3603843227028847, 3.3065551072359085], num_nbr [6.8277777777777775, 15.1]
Calculateing adj matrix using xy only...
Run 6: radius [2.8334697149693966, 3.3065551072359085], num_nbr [9.594444444444445, 15.1]
Calculateing adj matrix using xy only...
Run 7: radius [3.0700124111026525, 3.3065551072359085], num_nbr [9.594444444444445, 15.1]
Calculateing adj 

Run 63: radius [3.162277660168379, 3.1622776601683795], num_nbr [9.594444444444445, 15.1]
Calculateing adj matrix using xy only...
Run 64: radius [3.162277660168379, 3.1622776601683795], num_nbr [9.594444444444445, 15.1]
Calculateing adj matrix using xy only...
Run 65: radius [3.162277660168379, 3.1622776601683795], num_nbr [9.594444444444445, 15.1]
Calculateing adj matrix using xy only...
Run 66: radius [3.162277660168379, 3.1622776601683795], num_nbr [9.594444444444445, 15.1]
Calculateing adj matrix using xy only...
Run 67: radius [3.162277660168379, 3.1622776601683795], num_nbr [9.594444444444445, 15.1]
Calculateing adj matrix using xy only...
Run 68: radius [3.162277660168379, 3.1622776601683795], num_nbr [9.594444444444445, 15.1]
Calculateing adj matrix using xy only...
Run 69: radius [3.162277660168379, 3.1622776601683795], num_nbr [9.594444444444445, 15.1]
Calculateing adj matrix using xy only...
Run 70: radius [3.162277660168379, 3.1622776601683795], num_nbr [9.594444444444445,

In [30]:
target=1
#Set filtering criterials
min_in_group_fraction=0.8
min_in_out_group_ratio=1
min_fold_change=1.5
#Search radius such that each spot in the target domain has approximately 10 neighbors on average
adj_2d=spg.calculate_adj_matrix(x=x_array, y=y_array, histology=False)
start, end= np.quantile(adj_2d[adj_2d!=0],q=0.001), np.quantile(adj_2d[adj_2d!=0],q=0.1)
r=spg.search_radius(target_cluster=target, cell_id=adata.obs.index.tolist(), x=x_array, y=y_array, pred=adata.obs["pred"].tolist(), start=start, end=end, num_min=10, num_max=14,  max_run=100)
r = 3.1622776601683795
#Detect neighboring domains
print(r)
nbr_domians=spg.find_neighbor_clusters(target_cluster=target,
                                   cell_id=adata.obs.index.tolist(), 
                                   x=adata.obs["array_row"].tolist(), 
                                   y=adata.obs["array_col"].tolist(), 
                                   pred=adata.obs["pred"].tolist(),
                                   radius=r,
                                   ratio=1/2)


print(nbr_domians)
nbr_domians=nbr_domians[0:3]

de_genes_info=spg.rank_genes_groups(input_adata=adata,
                                target_cluster=target,
                                nbr_list=nbr_domians, 
                                label_col="pred", 
                                adj_nbr=True, 
                                log=True)
#Filter genes
de_genes_info=de_genes_info[(de_genes_info["pvals_adj"]<0.05)]
filtered_info_domain_0=de_genes_info
filtered_info_domain_0=filtered_info_domain_0[(filtered_info_domain_0["pvals_adj"]<0.05) &
                            (filtered_info_domain_0["in_out_group_ratio"]>min_in_out_group_ratio) &
                            (filtered_info_domain_0["in_group_fraction"]>min_in_group_fraction) &
                            (filtered_info_domain_0["fold_change"]>min_fold_change)]
filtered_info_domain_0=filtered_info_domain_0.sort_values(by="in_group_fraction", ascending=False)
filtered_info_domain_0["target_dmain"]=target
filtered_info_domain_0["neighbors"]=str(nbr_domians)
print("SVGs for domain ", str(target),":", filtered_info_domain_0["genes"].tolist())

Calculateing adj matrix using xy only...
Calculateing adj matrix using xy only...
Calculateing adj matrix using xy only...
Run 1: radius [1.4142135381698608, 16.552946090698242], num_nbr [1.0, 129.67117117117118]
Calculateing adj matrix using xy only...
Run 2: radius [1.4142135381698608, 8.983579814434052], num_nbr [1.0, 41.013513513513516]
Calculateing adj matrix using xy only...
Run 3: radius [1.4142135381698608, 5.198896676301956], num_nbr [1.0, 17.64864864864865]
Calculateing adj matrix using xy only...
recommended radius =  3.3065551072359085 num_nbr=10.175675675675675
3.1622776601683795
radius= 3.1622776601683795 average number of neighbors for each spot is 10.175675675675675
 Cluster 1 has neighbors:
Dmain  6 :  395
Dmain  5 :  274
Dmain  7 :  262
Dmain  3 :  238
Dmain  0 :  133
Dmain  8 :  123
Dmain  9 :  8
[6, 5, 7, 3, 0, 8, 9]
SVGs for domain  1 : ['PVALB', 'CST3', 'MTRNR2L12', 'MT-ND2', 'S100B', 'MT-CO1', 'MT-ND5', 'MT-ND3', 'MT-ND4', 'ALDOC', 'MT-ND1', 'C1orf61', 'PTGDS', '

In [31]:
target=2
#Set filtering criterials
min_in_group_fraction=0.8
min_in_out_group_ratio=1
min_fold_change=1.5
#Search radius such that each spot in the target domain has approximately 10 neighbors on average
adj_2d=spg.calculate_adj_matrix(x=x_array, y=y_array, histology=False)
start, end= np.quantile(adj_2d[adj_2d!=0],q=0.001), np.quantile(adj_2d[adj_2d!=0],q=0.1)
r=spg.search_radius(target_cluster=target, cell_id=adata.obs.index.tolist(), x=x_array, y=y_array, pred=adata.obs["pred"].tolist(), start=start, end=end, num_min=10, num_max=14,  max_run=100)
r = 3.1622776601683795
#Detect neighboring domains
print(r)
nbr_domians=spg.find_neighbor_clusters(target_cluster=target,
                                   cell_id=adata.obs.index.tolist(), 
                                   x=adata.obs["array_row"].tolist(), 
                                   y=adata.obs["array_col"].tolist(), 
                                   pred=adata.obs["pred"].tolist(),
                                   radius=r,
                                   ratio=1/2)


print(nbr_domians)
nbr_domians=nbr_domians[0:3]

de_genes_info=spg.rank_genes_groups(input_adata=adata,
                                target_cluster=target,
                                nbr_list=nbr_domians, 
                                label_col="pred", 
                                adj_nbr=True, 
                                log=True)
#Filter genes
de_genes_info=de_genes_info[(de_genes_info["pvals_adj"]<0.05)]
filtered_info_domain_0=de_genes_info
filtered_info_domain_0=filtered_info_domain_0[(filtered_info_domain_0["pvals_adj"]<0.05) &
                            (filtered_info_domain_0["in_out_group_ratio"]>min_in_out_group_ratio) &
                            (filtered_info_domain_0["in_group_fraction"]>min_in_group_fraction) &
                            (filtered_info_domain_0["fold_change"]>min_fold_change)]
filtered_info_domain_0=filtered_info_domain_0.sort_values(by="in_group_fraction", ascending=False)
filtered_info_domain_0["target_dmain"]=target
filtered_info_domain_0["neighbors"]=str(nbr_domians)
print("SVGs for domain ", str(target),":", filtered_info_domain_0["genes"].tolist())

Calculateing adj matrix using xy only...
Calculateing adj matrix using xy only...
Calculateing adj matrix using xy only...
Run 1: radius [1.4142135381698608, 16.552946090698242], num_nbr [1.0, 157.13698630136986]
Calculateing adj matrix using xy only...
Run 2: radius [1.4142135381698608, 8.983579814434052], num_nbr [1.0, 60.945205479452056]
Calculateing adj matrix using xy only...
Run 3: radius [1.4142135381698608, 5.198896676301956], num_nbr [1.0, 26.34246575342466]
Calculateing adj matrix using xy only...
Run 4: radius [1.4142135381698608, 3.3065551072359085], num_nbr [1.0, 14.363013698630137]
Calculateing adj matrix using xy only...
Run 5: radius [2.3603843227028847, 3.3065551072359085], num_nbr [6.821917808219178, 14.363013698630137]
Calculateing adj matrix using xy only...
Run 6: radius [2.8334697149693966, 3.3065551072359085], num_nbr [9.479452054794521, 14.363013698630137]
Calculateing adj matrix using xy only...
Run 7: radius [3.0700124111026525, 3.3065551072359085], num_nbr [9

Run 58: radius [3.162277660168379, 3.1622776601683795], num_nbr [9.479452054794521, 14.363013698630137]
Calculateing adj matrix using xy only...
Run 59: radius [3.162277660168379, 3.1622776601683795], num_nbr [9.479452054794521, 14.363013698630137]
Calculateing adj matrix using xy only...
Run 60: radius [3.162277660168379, 3.1622776601683795], num_nbr [9.479452054794521, 14.363013698630137]
Calculateing adj matrix using xy only...
Run 61: radius [3.162277660168379, 3.1622776601683795], num_nbr [9.479452054794521, 14.363013698630137]
Calculateing adj matrix using xy only...
Run 62: radius [3.162277660168379, 3.1622776601683795], num_nbr [9.479452054794521, 14.363013698630137]
Calculateing adj matrix using xy only...
Run 63: radius [3.162277660168379, 3.1622776601683795], num_nbr [9.479452054794521, 14.363013698630137]
Calculateing adj matrix using xy only...
Run 64: radius [3.162277660168379, 3.1622776601683795], num_nbr [9.479452054794521, 14.363013698630137]
Calculateing adj matrix us

In [32]:
target=3
#Set filtering criterials
min_in_group_fraction=0.8
min_in_out_group_ratio=1
min_fold_change=1.5
#Search radius such that each spot in the target domain has approximately 10 neighbors on average
adj_2d=spg.calculate_adj_matrix(x=x_array, y=y_array, histology=False)
start, end= np.quantile(adj_2d[adj_2d!=0],q=0.001), np.quantile(adj_2d[adj_2d!=0],q=0.1)
r=spg.search_radius(target_cluster=target, cell_id=adata.obs.index.tolist(), x=x_array, y=y_array, pred=adata.obs["pred"].tolist(), start=start, end=end, num_min=10, num_max=14,  max_run=100)
r = 3.1622776601683795
#Detect neighboring domains
print(r)
nbr_domians=spg.find_neighbor_clusters(target_cluster=target,
                                   cell_id=adata.obs.index.tolist(), 
                                   x=adata.obs["array_row"].tolist(), 
                                   y=adata.obs["array_col"].tolist(), 
                                   pred=adata.obs["pred"].tolist(),
                                   radius=r,
                                   ratio=1/2)


print(nbr_domians)
nbr_domians=nbr_domians[0:3]

de_genes_info=spg.rank_genes_groups(input_adata=adata,
                                target_cluster=target,
                                nbr_list=nbr_domians, 
                                label_col="pred", 
                                adj_nbr=True, 
                                log=True)
#Filter genes
de_genes_info=de_genes_info[(de_genes_info["pvals_adj"]<0.05)]
filtered_info_domain_0=de_genes_info
filtered_info_domain_0=filtered_info_domain_0[(filtered_info_domain_0["pvals_adj"]<0.05) &
                            (filtered_info_domain_0["in_out_group_ratio"]>min_in_out_group_ratio) &
                            (filtered_info_domain_0["in_group_fraction"]>min_in_group_fraction) &
                            (filtered_info_domain_0["fold_change"]>min_fold_change)]
filtered_info_domain_0=filtered_info_domain_0.sort_values(by="in_group_fraction", ascending=False)
filtered_info_domain_0["target_dmain"]=target
filtered_info_domain_0["neighbors"]=str(nbr_domians)
print("SVGs for domain ", str(target),":", filtered_info_domain_0["genes"].tolist())

Calculateing adj matrix using xy only...
Calculateing adj matrix using xy only...
Calculateing adj matrix using xy only...
Run 1: radius [1.4142135381698608, 16.552946090698242], num_nbr [1.0, 120.46706586826348]
Calculateing adj matrix using xy only...
Run 2: radius [1.4142135381698608, 8.983579814434052], num_nbr [1.0, 36.808383233532936]
Calculateing adj matrix using xy only...
Run 3: radius [1.4142135381698608, 5.198896676301956], num_nbr [1.0, 16.35928143712575]
Calculateing adj matrix using xy only...
Run 4: radius [3.3065551072359085, 5.198896676301956], num_nbr [9.988023952095809, 16.35928143712575]
Calculateing adj matrix using xy only...
recommended radius =  4.252725891768932 num_nbr=12.335329341317365
3.1622776601683795
radius= 3.1622776601683795 average number of neighbors for each spot is 9.988023952095809
 Cluster 3 has neighbors:
Dmain  6 :  239
Dmain  1 :  238
Dmain  8 :  181
Dmain  7 :  110
Dmain  5 :  108
Dmain  0 :  106
Dmain  9 :  14
[6, 1, 8, 7, 5, 0, 9]
SVGs for 

In [33]:
target=4
#Set filtering criterials
min_in_group_fraction=0.8
min_in_out_group_ratio=1
min_fold_change=1.5
#Search radius such that each spot in the target domain has approximately 10 neighbors on average
adj_2d=spg.calculate_adj_matrix(x=x_array, y=y_array, histology=False)
start, end= np.quantile(adj_2d[adj_2d!=0],q=0.001), np.quantile(adj_2d[adj_2d!=0],q=0.1)
r=spg.search_radius(target_cluster=target, cell_id=adata.obs.index.tolist(), x=x_array, y=y_array, pred=adata.obs["pred"].tolist(), start=start, end=end, num_min=10, num_max=14,  max_run=100)
r = 3.1622776601683795
#Detect neighboring domains
print(r)
nbr_domians=spg.find_neighbor_clusters(target_cluster=target,
                                   cell_id=adata.obs.index.tolist(), 
                                   x=adata.obs["array_row"].tolist(), 
                                   y=adata.obs["array_col"].tolist(), 
                                   pred=adata.obs["pred"].tolist(),
                                   radius=r,
                                   ratio=1/2)


print(nbr_domians)
nbr_domians=nbr_domians[0:3]

de_genes_info=spg.rank_genes_groups(input_adata=adata,
                                target_cluster=target,
                                nbr_list=nbr_domians, 
                                label_col="pred", 
                                adj_nbr=True, 
                                log=True)
#Filter genes
de_genes_info=de_genes_info[(de_genes_info["pvals_adj"]<0.05)]
filtered_info_domain_0=de_genes_info
filtered_info_domain_0=filtered_info_domain_0[(filtered_info_domain_0["pvals_adj"]<0.05) &
                            (filtered_info_domain_0["in_out_group_ratio"]>min_in_out_group_ratio) &
                            (filtered_info_domain_0["in_group_fraction"]>min_in_group_fraction) &
                            (filtered_info_domain_0["fold_change"]>min_fold_change)]
filtered_info_domain_0=filtered_info_domain_0.sort_values(by="in_group_fraction", ascending=False)
filtered_info_domain_0["target_dmain"]=target
filtered_info_domain_0["neighbors"]=str(nbr_domians)
print("SVGs for domain ", str(target),":", filtered_info_domain_0["genes"].tolist())

Calculateing adj matrix using xy only...
Calculateing adj matrix using xy only...
Calculateing adj matrix using xy only...
Run 1: radius [1.4142135381698608, 16.552946090698242], num_nbr [1.0, 175.69021739130434]
Calculateing adj matrix using xy only...
Run 2: radius [1.4142135381698608, 8.983579814434052], num_nbr [1.0, 71.60326086956522]
Calculateing adj matrix using xy only...
Run 3: radius [1.4142135381698608, 5.198896676301956], num_nbr [1.0, 28.831521739130434]
Calculateing adj matrix using xy only...
Run 4: radius [1.4142135381698608, 3.3065551072359085], num_nbr [1.0, 14.679347826086957]
Calculateing adj matrix using xy only...
Run 5: radius [2.3603843227028847, 3.3065551072359085], num_nbr [6.842391304347826, 14.679347826086957]
Calculateing adj matrix using xy only...
Run 6: radius [2.8334697149693966, 3.3065551072359085], num_nbr [9.467391304347826, 14.679347826086957]
Calculateing adj matrix using xy only...
Run 7: radius [3.0700124111026525, 3.3065551072359085], num_nbr [9

Run 58: radius [3.162277660168379, 3.1622776601683795], num_nbr [9.467391304347826, 14.679347826086957]
Calculateing adj matrix using xy only...
Run 59: radius [3.162277660168379, 3.1622776601683795], num_nbr [9.467391304347826, 14.679347826086957]
Calculateing adj matrix using xy only...
Run 60: radius [3.162277660168379, 3.1622776601683795], num_nbr [9.467391304347826, 14.679347826086957]
Calculateing adj matrix using xy only...
Run 61: radius [3.162277660168379, 3.1622776601683795], num_nbr [9.467391304347826, 14.679347826086957]
Calculateing adj matrix using xy only...
Run 62: radius [3.162277660168379, 3.1622776601683795], num_nbr [9.467391304347826, 14.679347826086957]
Calculateing adj matrix using xy only...
Run 63: radius [3.162277660168379, 3.1622776601683795], num_nbr [9.467391304347826, 14.679347826086957]
Calculateing adj matrix using xy only...
Run 64: radius [3.162277660168379, 3.1622776601683795], num_nbr [9.467391304347826, 14.679347826086957]
Calculateing adj matrix us

In [34]:
target=5
#Set filtering criterials
min_in_group_fraction=0.8
min_in_out_group_ratio=1
min_fold_change=1.5
#Search radius such that each spot in the target domain has approximately 10 neighbors on average
adj_2d=spg.calculate_adj_matrix(x=x_array, y=y_array, histology=False)
start, end= np.quantile(adj_2d[adj_2d!=0],q=0.001), np.quantile(adj_2d[adj_2d!=0],q=0.1)
r=spg.search_radius(target_cluster=target, cell_id=adata.obs.index.tolist(), x=x_array, y=y_array, pred=adata.obs["pred"].tolist(), start=start, end=end, num_min=10, num_max=14,  max_run=100)
r = 3.1622776601683795
#Detect neighboring domains
print(r)
nbr_domians=spg.find_neighbor_clusters(target_cluster=target,
                                   cell_id=adata.obs.index.tolist(), 
                                   x=adata.obs["array_row"].tolist(), 
                                   y=adata.obs["array_col"].tolist(), 
                                   pred=adata.obs["pred"].tolist(),
                                   radius=r,
                                   ratio=1/2)

print(nbr_domians)
nbr_domians=nbr_domians[0:3]

de_genes_info=spg.rank_genes_groups(input_adata=adata,
                                target_cluster=target,
                                nbr_list=nbr_domians, 
                                label_col="pred", 
                                adj_nbr=True, 
                                log=True)
#Filter genes
de_genes_info=de_genes_info[(de_genes_info["pvals_adj"]<0.05)]
filtered_info_domain_0=de_genes_info
filtered_info_domain_0=filtered_info_domain_0[(filtered_info_domain_0["pvals_adj"]<0.05) &
                            (filtered_info_domain_0["in_out_group_ratio"]>min_in_out_group_ratio) &
                            (filtered_info_domain_0["in_group_fraction"]>min_in_group_fraction) &
                            (filtered_info_domain_0["fold_change"]>min_fold_change)]
filtered_info_domain_0=filtered_info_domain_0.sort_values(by="in_group_fraction", ascending=False)
filtered_info_domain_0["target_dmain"]=target
filtered_info_domain_0["neighbors"]=str(nbr_domians)
print("SVGs for domain ", str(target),":", filtered_info_domain_0["genes"].tolist())

Calculateing adj matrix using xy only...
Calculateing adj matrix using xy only...
Calculateing adj matrix using xy only...
Run 1: radius [1.4142135381698608, 16.552946090698242], num_nbr [1.0, 159.6]
Calculateing adj matrix using xy only...
Run 2: radius [1.4142135381698608, 8.983579814434052], num_nbr [1.0, 47.3]
Calculateing adj matrix using xy only...
Run 3: radius [1.4142135381698608, 5.198896676301956], num_nbr [1.0, 17.266666666666666]
Calculateing adj matrix using xy only...
Run 4: radius [3.3065551072359085, 5.198896676301956], num_nbr [9.286666666666667, 17.266666666666666]
Calculateing adj matrix using xy only...
recommended radius =  4.252725891768932 num_nbr=12.026666666666667
3.1622776601683795
radius= 3.1622776601683795 average number of neighbors for each spot is 9.286666666666667
 Cluster 5 has neighbors:
Dmain  1 :  274
Dmain  3 :  108
Dmain  7 :  95
Dmain  8 :  41
[1, 3, 7, 8]
SVGs for domain  5 : ['MTRNR2L12', 'MT-ND4L', 'MT-ND5', 'RPS27', 'MTRNR2L1', 'MTRNR2L8', 'MT

In [35]:
target=6
#Set filtering criterials
min_in_group_fraction=0.8
min_in_out_group_ratio=1
min_fold_change=1.5
#Search radius such that each spot in the target domain has approximately 10 neighbors on average
adj_2d=spg.calculate_adj_matrix(x=x_array, y=y_array, histology=False)
start, end= np.quantile(adj_2d[adj_2d!=0],q=0.001), np.quantile(adj_2d[adj_2d!=0],q=0.1)
r=spg.search_radius(target_cluster=target, cell_id=adata.obs.index.tolist(), x=x_array, y=y_array, pred=adata.obs["pred"].tolist(), start=start, end=end, num_min=10, num_max=14,  max_run=100)
r = 3.1622776601683795
#Detect neighboring domains
print(r)
nbr_domians=spg.find_neighbor_clusters(target_cluster=target,
                                   cell_id=adata.obs.index.tolist(), 
                                   x=adata.obs["array_row"].tolist(), 
                                   y=adata.obs["array_col"].tolist(), 
                                   pred=adata.obs["pred"].tolist(),
                                   radius=r,
                                   ratio=1/2)


print(nbr_domians)
nbr_domians=nbr_domians[0:3]

de_genes_info=spg.rank_genes_groups(input_adata=adata,
                                target_cluster=target,
                                nbr_list=nbr_domians, 
                                label_col="pred", 
                                adj_nbr=True, 
                                log=True)
#Filter genes
de_genes_info=de_genes_info[(de_genes_info["pvals_adj"]<0.05)]
filtered_info_domain_0=de_genes_info
filtered_info_domain_0=filtered_info_domain_0[(filtered_info_domain_0["pvals_adj"]<0.05) &
                            (filtered_info_domain_0["in_out_group_ratio"]>min_in_out_group_ratio) &
                            (filtered_info_domain_0["in_group_fraction"]>min_in_group_fraction) &
                            (filtered_info_domain_0["fold_change"]>min_fold_change)]
filtered_info_domain_0=filtered_info_domain_0.sort_values(by="in_group_fraction", ascending=False)
filtered_info_domain_0["target_dmain"]=target
filtered_info_domain_0["neighbors"]=str(nbr_domians)
print("SVGs for domain ", str(target),":", filtered_info_domain_0["genes"].tolist())

Calculateing adj matrix using xy only...
Calculateing adj matrix using xy only...
Calculateing adj matrix using xy only...
Run 1: radius [1.4142135381698608, 16.552946090698242], num_nbr [1.0, 148.5219512195122]
Calculateing adj matrix using xy only...
Run 2: radius [1.4142135381698608, 8.983579814434052], num_nbr [1.0, 53.72682926829268]
Calculateing adj matrix using xy only...
Run 3: radius [1.4142135381698608, 5.198896676301956], num_nbr [1.0, 23.434146341463414]
Calculateing adj matrix using xy only...
recommended radius =  3.3065551072359085 num_nbr=12.897560975609757
3.1622776601683795
radius= 3.1622776601683795 average number of neighbors for each spot is 12.897560975609757
 Cluster 6 has neighbors:
Dmain  1 :  395
Dmain  0 :  337
Dmain  4 :  256
Dmain  3 :  239
Dmain  7 :  229
Dmain  2 :  221
Dmain  8 :  136
Dmain  5 :  99
Dmain  9 :  7
[1, 0, 4, 3, 7, 2, 8, 5, 9]
SVGs for domain  6 : []


In [36]:
target=7
#Set filtering criterials
min_in_group_fraction=0.8
min_in_out_group_ratio=1
min_fold_change=1.5
#Search radius such that each spot in the target domain has approximately 10 neighbors on average
adj_2d=spg.calculate_adj_matrix(x=x_array, y=y_array, histology=False)
start, end= np.quantile(adj_2d[adj_2d!=0],q=0.001), np.quantile(adj_2d[adj_2d!=0],q=0.1)
r=spg.search_radius(target_cluster=target, cell_id=adata.obs.index.tolist(), x=x_array, y=y_array, pred=adata.obs["pred"].tolist(), start=start, end=end, num_min=10, num_max=14,  max_run=100)
r = 3.1622776601683795
#Detect neighboring domains
print(r)
nbr_domians=spg.find_neighbor_clusters(target_cluster=target,
                                   cell_id=adata.obs.index.tolist(), 
                                   x=adata.obs["array_row"].tolist(), 
                                   y=adata.obs["array_col"].tolist(), 
                                   pred=adata.obs["pred"].tolist(),
                                   radius=r,
                                   ratio=1/2)


print(nbr_domians)
nbr_domians=nbr_domians[0:3]

de_genes_info=spg.rank_genes_groups(input_adata=adata,
                                target_cluster=target,
                                nbr_list=nbr_domians, 
                                label_col="pred", 
                                adj_nbr=True, 
                                log=True)
#Filter genes
de_genes_info=de_genes_info[(de_genes_info["pvals_adj"]<0.05)]
filtered_info_domain_0=de_genes_info
filtered_info_domain_0=filtered_info_domain_0[(filtered_info_domain_0["pvals_adj"]<0.05) &
                            (filtered_info_domain_0["in_out_group_ratio"]>min_in_out_group_ratio) &
                            (filtered_info_domain_0["in_group_fraction"]>min_in_group_fraction) &
                            (filtered_info_domain_0["fold_change"]>min_fold_change)]
filtered_info_domain_0=filtered_info_domain_0.sort_values(by="in_group_fraction", ascending=False)
filtered_info_domain_0["target_dmain"]=target
filtered_info_domain_0["neighbors"]=str(nbr_domians)
print("SVGs for domain ", str(target),":", filtered_info_domain_0["genes"].tolist())

Calculateing adj matrix using xy only...
Calculateing adj matrix using xy only...
Calculateing adj matrix using xy only...
Run 1: radius [1.4142135381698608, 16.552946090698242], num_nbr [1.0, 125.33884297520662]
Calculateing adj matrix using xy only...
Run 2: radius [1.4142135381698608, 8.983579814434052], num_nbr [1.0, 41.429752066115704]
Calculateing adj matrix using xy only...
Run 3: radius [1.4142135381698608, 5.198896676301956], num_nbr [1.0, 18.31404958677686]
Calculateing adj matrix using xy only...
recommended radius =  3.3065551072359085 num_nbr=10.884297520661157
3.1622776601683795
radius= 3.1622776601683795 average number of neighbors for each spot is 10.884297520661157
 Cluster 7 has neighbors:
Dmain  1 :  262
Dmain  6 :  229
Dmain  0 :  131
Dmain  3 :  110
Dmain  4 :  99
Dmain  5 :  95
Dmain  8 :  64
Dmain  9 :  7
[1, 6, 0, 3, 4, 5, 8, 9]
SVGs for domain  7 : ['MT3', 'S100B', 'SLC1A3', 'CST3', 'ACTG1', 'SEPTIN4', 'NDRG2', 'PTGDS', 'APOE', 'CPE', 'TUBB2B', 'C1orf61', 'GLUL

In [37]:
target=8
#Set filtering criterials
min_in_group_fraction=0.8
min_in_out_group_ratio=1
min_fold_change=1.5
#Search radius such that each spot in the target domain has approximately 10 neighbors on average
adj_2d=spg.calculate_adj_matrix(x=x_array, y=y_array, histology=False)
start, end= np.quantile(adj_2d[adj_2d!=0],q=0.001), np.quantile(adj_2d[adj_2d!=0],q=0.1)
r=spg.search_radius(target_cluster=target, cell_id=adata.obs.index.tolist(), x=x_array, y=y_array, pred=adata.obs["pred"].tolist(), start=start, end=end, num_min=10, num_max=14,  max_run=100)
r = 3.1622776601683795
#Detect neighboring domains
print(r)
nbr_domians=spg.find_neighbor_clusters(target_cluster=target,
                                   cell_id=adata.obs.index.tolist(), 
                                   x=adata.obs["array_row"].tolist(), 
                                   y=adata.obs["array_col"].tolist(), 
                                   pred=adata.obs["pred"].tolist(),
                                   radius=r,
                                   ratio=1/2)


print(nbr_domians)
nbr_domians=nbr_domians[0:3]

de_genes_info=spg.rank_genes_groups(input_adata=adata,
                                target_cluster=target,
                                nbr_list=nbr_domians, 
                                label_col="pred", 
                                adj_nbr=True, 
                                log=True)
#Filter genes
de_genes_info=de_genes_info[(de_genes_info["pvals_adj"]<0.05)]
filtered_info_domain_0=de_genes_info
filtered_info_domain_0=filtered_info_domain_0[(filtered_info_domain_0["pvals_adj"]<0.05) &
                            (filtered_info_domain_0["in_out_group_ratio"]>min_in_out_group_ratio) &
                            (filtered_info_domain_0["in_group_fraction"]>min_in_group_fraction) &
                            (filtered_info_domain_0["fold_change"]>min_fold_change)]
filtered_info_domain_0=filtered_info_domain_0.sort_values(by="in_group_fraction", ascending=False)
filtered_info_domain_0["target_dmain"]=target
filtered_info_domain_0["neighbors"]=str(nbr_domians)
print("SVGs for domain ", str(target),":", filtered_info_domain_0["genes"].tolist())

Calculateing adj matrix using xy only...
Calculateing adj matrix using xy only...
Calculateing adj matrix using xy only...
Run 1: radius [1.4142135381698608, 16.552946090698242], num_nbr [1.0, 129.0126582278481]
Calculateing adj matrix using xy only...
Run 2: radius [1.4142135381698608, 8.983579814434052], num_nbr [1.0, 43.45569620253165]
Calculateing adj matrix using xy only...
Run 3: radius [1.4142135381698608, 5.198896676301956], num_nbr [1.0, 18.72151898734177]
Calculateing adj matrix using xy only...
recommended radius =  3.3065551072359085 num_nbr=11.012658227848101
3.1622776601683795
radius= 3.1622776601683795 average number of neighbors for each spot is 11.012658227848101
 Cluster 8 has neighbors:
Dmain  3 :  181
Dmain  6 :  136
Dmain  1 :  123
Dmain  7 :  64
[3, 6, 1, 7]
SVGs for domain  8 : ['ALDOC', 'YWHAH', 'PCP4', 'SLC1A6', 'NSG1', 'CA8', 'ITPR1', 'DNER', 'CALB1', 'ATP2A3']


In [38]:
target=9
#Set filtering criterials
min_in_group_fraction=0.8
min_in_out_group_ratio=1
min_fold_change=1.5
#Search radius such that each spot in the target domain has approximately 10 neighbors on average
adj_2d=spg.calculate_adj_matrix(x=x_array, y=y_array, histology=False)
start, end= np.quantile(adj_2d[adj_2d!=0],q=0.001), np.quantile(adj_2d[adj_2d!=0],q=0.1)
r=spg.search_radius(target_cluster=target, cell_id=adata.obs.index.tolist(), x=x_array, y=y_array, pred=adata.obs["pred"].tolist(), start=start, end=end, num_min=10, num_max=14,  max_run=100)
r = 3.1622776601683795
#Detect neighboring domains
print(r)
nbr_domians=spg.find_neighbor_clusters(target_cluster=target,
                                   cell_id=adata.obs.index.tolist(), 
                                   x=adata.obs["array_row"].tolist(), 
                                   y=adata.obs["array_col"].tolist(), 
                                   pred=adata.obs["pred"].tolist(),
                                   radius=r,
                                   ratio=1/2)


print(nbr_domians)
nbr_domians=nbr_domians[0:3]

de_genes_info=spg.rank_genes_groups(input_adata=adata,
                                target_cluster=target,
                                nbr_list=nbr_domians, 
                                label_col="pred", 
                                adj_nbr=True, 
                                log=True)
#Filter genes
de_genes_info=de_genes_info[(de_genes_info["pvals_adj"]<0.05)]
filtered_info_domain_0=de_genes_info
filtered_info_domain_0=filtered_info_domain_0[(filtered_info_domain_0["pvals_adj"]<0.05) &
                            (filtered_info_domain_0["in_out_group_ratio"]>min_in_out_group_ratio) &
                            (filtered_info_domain_0["in_group_fraction"]>min_in_group_fraction) &
                            (filtered_info_domain_0["fold_change"]>min_fold_change)]
filtered_info_domain_0=filtered_info_domain_0.sort_values(by="in_group_fraction", ascending=False)
filtered_info_domain_0["target_dmain"]=target
filtered_info_domain_0["neighbors"]=str(nbr_domians)
print("SVGs for domain ", str(target),":", filtered_info_domain_0["genes"].tolist())

Calculateing adj matrix using xy only...
Calculateing adj matrix using xy only...
Calculateing adj matrix using xy only...
Run 1: radius [1.4142135381698608, 16.552946090698242], num_nbr [1.0, 124.8]
Calculateing adj matrix using xy only...
Run 2: radius [1.4142135381698608, 8.983579814434052], num_nbr [1.0, 39.8]
Calculateing adj matrix using xy only...
Run 3: radius [1.4142135381698608, 5.198896676301956], num_nbr [1.0, 17.8]
Calculateing adj matrix using xy only...
recommended radius =  3.3065551072359085 num_nbr=10.8
3.1622776601683795
radius= 3.1622776601683795 average number of neighbors for each spot is 10.8
 Cluster 9 has neighbors:
No neighbor domain found, try bigger radius or smaller ratio.
None


TypeError: 'NoneType' object is not subscriptable